In [ ]:
import itertools
import pulp

def find_multiset(n, v_dict, m):
    """
    Solve for a multiset M in F2^n satisfying divisibility properties:
      - For each beta in F2^n \ {0}, sum_x (-1)^{<beta,x>} * mu[x] = 2^{v_beta+1} * d_beta
      - sum_x mu[x] = 2^m
    Additionally, enforce each d_beta to be an odd integer.

    Args:
        n (int): dimension of the F2 vector space
        v_dict (dict[int, int]): map from beta (encoded as int 1..2^n-1) to v_beta
        m (int): exponent for total multiset size (|M| = 2^m)

    Returns:
        (dict[int,int], dict[int,int]): solutions for mu(x) and d_beta
    """
    
    
    # Create MILP problem: minimize max multiplicity for uniformity
    prob = pulp.LpProblem("Multiset_Divisibility_OddDBeta", pulp.LpMinimize)

    # Variables: mu[x] >= 0 integer multiplicities
    mu = {x: pulp.LpVariable(f"mu_{x}", lowBound=0, cat='Integer')
          for x in range(2**n)}
    # d_beta integer (no bounds)
    d = {beta: pulp.LpVariable(f"d_{beta}", lowBound=None, cat='Integer')
         for beta in range(1, 2**n)}

    # Constraint: total size = 2^m
    prob += sum(mu[x] for x in mu) == 2**m, "TotalSize"

#     # Constraints per beta != 0
#     for beta in range(1, 2**n):
#         v = v_dict.get(beta, 0)
#         # Divisibility: sum_x (-1)^{<beta,x>} mu[x] = 2^{v+1} * d_beta
#         expr = pulp.lpSum(((-1)**(bin(x & beta).count('1') & 1)) * mu[x]
#                           for x in mu)
#         prob += expr == 2**(v + 1) * (2 * d[beta] + 1), f"Divisible_beta_{beta}"

    # Constraints per beta != 0
#     for beta in v_dict.keys():
    for beta in d:
        v = v_dict.get(beta, 0)
        # Divisibility: sum_x (-1)^{<beta,x>} mu[x] = 2^{v+1} * d_beta
        expr = pulp.lpSum(((-1)**((x & beta).bit_count() & 1)) * mu[x]
                          for x in mu)
        prob += expr == 2**(v + 1) * (2 * d[beta] + 1), f"Divisible_beta_{beta}"
    
     # Auxiliary for uniformity: minimize the maximum mu[x]
#     T = pulp.LpVariable("T", lowBound=0, cat='Integer')
#     for x in mu:
#         prob += mu[x] <= T
#     prob += T, "Minimize_max_mu"
    print(prob)
    # Optional objective: find any feasible solution
    prob += 0, "Objective"

    # Solve with GLPK (or swap to CBC with path if desired)
    result = prob.solve(pulp.GLPK_CMD(msg=True))
    if pulp.LpStatus[result] != 'Optimal':
        raise ValueError(f"No optimal solution found: status {pulp.LpStatus[result]}")

    # Extract solutions
    mu_sol = {x: int(mu[x].value()) for x in mu}
    d_sol = {beta: int(d[beta].value()) for beta in v_dict.keys()}
    return mu_sol, d_sol


if __name__ == "__main__":
    # Example usage for n=4
    n = 4
    # Specify some v_beta values
    v_dict = {1:1}
    
    
#     for i in range(1,15):
#         v_dict[i]  = 3
        
#     v_dict[15]  = 3
    m = 5  # total size of M = 2^m

    mu_solution, d_solution = find_multiset(n, v_dict, m)
    print("Mu(x) values:", mu_solution)
    print("Odd d_beta values:", d_solution)

Multiset_Divisibility_OddDBeta:
MINIMIZE
None
SUBJECT TO
TotalSize: mu_0 + mu_1 + mu_10 + mu_11 + mu_12 + mu_13 + mu_14 + mu_15 + mu_2
 + mu_3 + mu_4 + mu_5 + mu_6 + mu_7 + mu_8 + mu_9 = 32

Divisible_beta_1: - 8 d_1 + mu_0 - mu_1 + mu_10 - mu_11 + mu_12 - mu_13
 + mu_14 - mu_15 + mu_2 - mu_3 + mu_4 - mu_5 + mu_6 - mu_7 + mu_8 - mu_9 = 4

Divisible_beta_2: - 4 d_2 + mu_0 + mu_1 - mu_10 - mu_11 + mu_12 + mu_13
 - mu_14 - mu_15 - mu_2 - mu_3 + mu_4 + mu_5 - mu_6 - mu_7 + mu_8 + mu_9 = 2

Divisible_beta_3: - 4 d_3 + mu_0 - mu_1 - mu_10 + mu_11 + mu_12 - mu_13
 - mu_14 + mu_15 - mu_2 + mu_3 + mu_4 - mu_5 - mu_6 + mu_7 + mu_8 - mu_9 = 2

Divisible_beta_4: - 4 d_4 + mu_0 + mu_1 + mu_10 + mu_11 - mu_12 - mu_13
 - mu_14 - mu_15 + mu_2 + mu_3 - mu_4 - mu_5 - mu_6 - mu_7 + mu_8 + mu_9 = 2

Divisible_beta_5: - 4 d_5 + mu_0 - mu_1 + mu_10 - mu_11 - mu_12 + mu_13
 - mu_14 + mu_15 + mu_2 - mu_3 - mu_4 + mu_5 - mu_6 + mu_7 + mu_8 - mu_9 = 2

Divisible_beta_6: - 4 d_6 + mu_0 + mu_1 - mu_10 - mu_11 - m

+2685259: mip =     not found yet >=   0.000000000e+00        (34057; 267130)
+2710123: mip =     not found yet >=   0.000000000e+00        (33875; 270321)
+2728029: mip =     not found yet >=   0.000000000e+00        (33669; 272540)
+2743491: mip =     not found yet >=   0.000000000e+00        (33651; 274401)
+2757549: mip =     not found yet >=   0.000000000e+00        (34625; 275815)
+2772998: mip =     not found yet >=   0.000000000e+00        (35512; 277459)
+2790361: mip =     not found yet >=   0.000000000e+00        (36403; 279120)
+2804381: mip =     not found yet >=   0.000000000e+00        (37110; 280682)
Time used: 300.0 secs.  Memory used: 67.6 Mb.
+2813402: mip =     not found yet >=   0.000000000e+00        (37628; 281578)
+2823726: mip =     not found yet >=   0.000000000e+00        (37921; 282829)
+2834855: mip =     not found yet >=   0.000000000e+00        (38345; 284083)
+2847277: mip =     not found yet >=   0.000000000e+00        (38642; 285630)
+2859899: mip =   

+4117923: mip =     not found yet >=   0.000000000e+00        (48676; 415592)
+4129518: mip =     not found yet >=   0.000000000e+00        (48640; 417594)
+4141491: mip =     not found yet >=   0.000000000e+00        (48700; 418868)
Time used: 780.0 secs.  Memory used: 94.1 Mb.
+4153073: mip =     not found yet >=   0.000000000e+00        (48728; 420343)
+4162796: mip =     not found yet >=   0.000000000e+00        (48735; 421819)
+4171789: mip =     not found yet >=   0.000000000e+00        (48633; 423583)
+4180265: mip =     not found yet >=   0.000000000e+00        (48704; 424761)
+4189176: mip =     not found yet >=   0.000000000e+00        (48707; 426237)
+4199473: mip =     not found yet >=   0.000000000e+00        (48807; 427570)
+4211323: mip =     not found yet >=   0.000000000e+00        (48912; 428822)
+4222890: mip =     not found yet >=   0.000000000e+00        (49059; 429921)
+4232842: mip =     not found yet >=   0.000000000e+00        (49266; 430982)
+4247568: mip =   

+5086913: mip =     not found yet >=   0.000000000e+00        (59835; 546175)
+5096165: mip =     not found yet >=   0.000000000e+00        (59877; 547339)
+5105833: mip =     not found yet >=   0.000000000e+00        (60005; 548360)
+5114876: mip =     not found yet >=   0.000000000e+00        (59937; 549612)
+5122312: mip =     not found yet >=   0.000000000e+00        (60074; 550665)
+5130264: mip =     not found yet >=   0.000000000e+00        (60126; 551695)
+5137035: mip =     not found yet >=   0.000000000e+00        (60265; 552600)
+5145810: mip =     not found yet >=   0.000000000e+00        (60209; 553774)
+5153923: mip =     not found yet >=   0.000000000e+00        (60275; 554898)
+5164314: mip =     not found yet >=   0.000000000e+00        (60368; 556102)
+5172449: mip =     not found yet >=   0.000000000e+00        (60442; 557108)
Time used: 1320.1 secs.  Memory used: 115.9 Mb.
+5179906: mip =     not found yet >=   0.000000000e+00        (60407; 558242)
+5187286: mip = 

+5839201: mip =     not found yet >=   0.000000000e+00        (86518; 629963)
+5844728: mip =     not found yet >=   0.000000000e+00        (86712; 630579)
+5851092: mip =     not found yet >=   0.000000000e+00        (86878; 631216)
+5857236: mip =     not found yet >=   0.000000000e+00        (87072; 631854)
+5863076: mip =     not found yet >=   0.000000000e+00        (87206; 632563)
+5868287: mip =     not found yet >=   0.000000000e+00        (87420; 633249)
Time used: 1800.1 secs.  Memory used: 151.1 Mb.
+5874805: mip =     not found yet >=   0.000000000e+00        (87550; 633852)
+5881026: mip =     not found yet >=   0.000000000e+00        (87731; 634513)
+5887185: mip =     not found yet >=   0.000000000e+00        (87955; 635086)
+5893687: mip =     not found yet >=   0.000000000e+00        (88078; 635755)
+5899661: mip =     not found yet >=   0.000000000e+00        (88272; 636372)
+5905838: mip =     not found yet >=   0.000000000e+00        (88500; 636971)
+5912277: mip = 

...

+15304605: mip =     not found yet >=   0.000000000e+00        (231702; 1820405)
+15306217: mip =     not found yet >=   0.000000000e+00        (231733; 1820687)
+15307848: mip =     not found yet >=   0.000000000e+00        (231744; 1820996)
+15310049: mip =     not found yet >=   0.000000000e+00        (231760; 1821324)
+15311888: mip =     not found yet >=   0.000000000e+00        (231794; 1821596)
+15313450: mip =     not found yet >=   0.000000000e+00        (231801; 1821910)
+15315197: mip =     not found yet >=   0.000000000e+00        (231820; 1822210)
+15316791: mip =     not found yet >=   0.000000000e+00        (231829; 1822509)
+15319350: mip =     not found yet >=   0.000000000e+00        (231833; 1822827)
+15322039: mip =     not found yet >=   0.000000000e+00        (231804; 1823177)
Time used: 16502.8 secs.  Memory used: 399.8 Mb.
+15324798: mip =     not found yet >=   0.000000000e+00        (231837; 1823476)
+15326583: mip =     not found yet >=   0.000000000e+00     

+15507175: mip =     not found yet >=   0.000000000e+00        (233227; 1850017)
+15508758: mip =     not found yet >=   0.000000000e+00        (233296; 1850208)
+15510658: mip =     not found yet >=   0.000000000e+00        (233353; 1850414)
+15512273: mip =     not found yet >=   0.000000000e+00        (233418; 1850617)
+15513815: mip =     not found yet >=   0.000000000e+00        (233469; 1850836)
+15515401: mip =     not found yet >=   0.000000000e+00        (233522; 1851042)
+15517550: mip =     not found yet >=   0.000000000e+00        (233570; 1851252)
+15520004: mip =     not found yet >=   0.000000000e+00        (233659; 1851452)
+15522068: mip =     not found yet >=   0.000000000e+00        (233656; 1851703)
Time used: 16982.9 secs.  Memory used: 403.1 Mb.
+15524443: mip =     not found yet >=   0.000000000e+00        (233708; 1851947)
+15526336: mip =     not found yet >=   0.000000000e+00        (233751; 1852209)
+15528830: mip =     not found yet >=   0.000000000e+00     

+15718131: mip =     not found yet >=   0.000000000e+00        (239523; 1874331)
+15720355: mip =     not found yet >=   0.000000000e+00        (239621; 1874582)
+15722703: mip =     not found yet >=   0.000000000e+00        (239724; 1874820)
+15724918: mip =     not found yet >=   0.000000000e+00        (239833; 1875052)
+15726687: mip =     not found yet >=   0.000000000e+00        (239954; 1875277)
+15728403: mip =     not found yet >=   0.000000000e+00        (240071; 1875522)
+15730120: mip =     not found yet >=   0.000000000e+00        (240148; 1875775)
Time used: 17463.0 secs.  Memory used: 413.1 Mb.
+15731532: mip =     not found yet >=   0.000000000e+00        (240184; 1876030)
+15733954: mip =     not found yet >=   0.000000000e+00        (240238; 1876276)
+15735706: mip =     not found yet >=   0.000000000e+00        (240330; 1876479)
+15737906: mip =     not found yet >=   0.000000000e+00        (240427; 1876699)
+15740310: mip =     not found yet >=   0.000000000e+00     

+15929520: mip =     not found yet >=   0.000000000e+00        (245579; 1899278)
+15932273: mip =     not found yet >=   0.000000000e+00        (245658; 1899502)
+15935007: mip =     not found yet >=   0.000000000e+00        (245738; 1899742)
+15937612: mip =     not found yet >=   0.000000000e+00        (245766; 1900020)
+15940176: mip =     not found yet >=   0.000000000e+00        (245800; 1900302)
+15943402: mip =     not found yet >=   0.000000000e+00        (245849; 1900573)
Time used: 17943.2 secs.  Memory used: 421.9 Mb.
+15946097: mip =     not found yet >=   0.000000000e+00        (245908; 1900822)
+15948308: mip =     not found yet >=   0.000000000e+00        (246000; 1901040)
+15950136: mip =     not found yet >=   0.000000000e+00        (246070; 1901287)
+15952500: mip =     not found yet >=   0.000000000e+00        (246127; 1901531)
+15955522: mip =     not found yet >=   0.000000000e+00        (246214; 1901752)
+15957762: mip =     not found yet >=   0.000000000e+00     

+16137263: mip =     not found yet >=   0.000000000e+00        (251265; 1923467)
+16139344: mip =     not found yet >=   0.000000000e+00        (251363; 1923706)
+16141162: mip =     not found yet >=   0.000000000e+00        (251416; 1923912)
+16142622: mip =     not found yet >=   0.000000000e+00        (251456; 1924129)
Time used: 18423.3 secs.  Memory used: 430.7 Mb.
+16144843: mip =     not found yet >=   0.000000000e+00        (251500; 1924375)
+16146576: mip =     not found yet >=   0.000000000e+00        (251525; 1924637)
+16148386: mip =     not found yet >=   0.000000000e+00        (251613; 1924868)
+16150203: mip =     not found yet >=   0.000000000e+00        (251667; 1925077)
+16152062: mip =     not found yet >=   0.000000000e+00        (251709; 1925317)
+16153479: mip =     not found yet >=   0.000000000e+00        (251743; 1925553)
+16155876: mip =     not found yet >=   0.000000000e+00        (251745; 1925836)
+16158728: mip =     not found yet >=   0.000000000e+00     